# Imports

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sb
import time
# import mat4py
import scipy.io 
import sklearn
from sklearn import preprocessing
import plotly.express as px
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans 
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.fft import fft, ifft
from scipy.signal import lfilter,butter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [ ]:
!pip install tensorflow
!pip install antropy

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


In [ ]:
import antropy as ant
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# from keras.utils import to_categorical

In [ ]:
from tensorflow.keras.utils import to_categorical

# Data loading

In [ ]:
#TODO

#Input data loading

In [ ]:
! ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
real_signal_filename = 'Dataset1.mat'
attack_signal_filename = 'sampleAttack.mat'


In [ ]:

def load_realsignal_from_path(fpath):  #load real signal from path
    mat = scipy.io.loadmat(fpath)
    raw_data = mat['Raw_Data']
    total_rows = []
    for i in range(raw_data.shape[0]):
        for j in range(raw_data.shape[1]):
            total_rows.extend(np.array_split(raw_data[i][j], 4))
    raw_df = pd.DataFrame(total_rows)
    return raw_df

def load_attacksignal_from_path(fpath): #load attack signal from path
    mat = scipy.io.loadmat(fpath)
    raw_data = mat['attackVectors']
    total_rows = []
    for i in range(raw_data.shape[0]):
        for j in range(raw_data.shape[1]):
            for k in range(raw_data.shape[2]):
                total_rows.append(raw_data[i][j][k])
    raw_df = pd.DataFrame(total_rows)
    return raw_df

def load_inputsignal_from_path(fpath): #load input signal from path
    # mat = scipy.io.loadmat(fpath)
    # raw_data = mat['attackVectors']
    # total_rows = []
    # for i in range(raw_data.shape[0]):
    #     for j in range(raw_data.shape[1]):
    #         for k in range(raw_data.shape[2]):
    #             total_rows.append(raw_data[i][j][k])
    # raw_df = pd.DataFrame(total_rows)
    # return raw_df
    input_df = pd.read_csv("input.csv")
    



In [ ]:
real_df = load_realsignal_from_path(real_signal_filename)

In [ ]:
attack_df = load_attacksignal_from_path(attack_signal_filename)

In [ ]:
real_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,-12,-19,-30,-53,-65,-59,-46,-42,-65,-74,...,-49,-33,-40,-31,-45,-65,-43,-32,-23,-22
1,-4,23,39,23,22,5,8,26,26,10,...,-48,-33,-45,-51,-27,-31,-42,-63,-65,-39
2,-31,-48,-67,-88,-88,-49,-34,-36,-22,-23,...,6,2,1,12,11,3,48,70,68,55
3,73,54,69,66,37,19,27,54,45,1,...,1,44,61,64,36,28,16,26,11,-5
4,-63,-40,-15,-24,-15,-26,-45,-33,-42,5,...,-30,-33,-45,-56,-49,-48,-63,-65,-63,-54


In [ ]:
! dir

 Volume in drive D is New Volume
 Volume Serial Number is 12A8-7A11

 Directory of d:\mobilecompfinal

12/03/2021  09:50 PM    <DIR>          .
12/03/2021  09:50 PM    <DIR>          ..
12/03/2021  07:32 PM    <DIR>          code
11/27/2021  06:17 PM         8,255,832 Dataset1.mat
12/03/2021  07:32 PM    <DIR>          datasets
12/03/2021  09:51 PM    <DIR>          firstbranch
12/03/2021  09:50 PM    <DIR>          git
12/03/2021  07:33 PM    <DIR>          inputs
11/28/2021  10:41 PM        70,972,232 sampleAttack.mat
12/08/2021  08:27 PM           692,005 ver1 (1).ipynb
               3 File(s)     79,920,069 bytes
               7 Dir(s)  78,345,629,696 bytes free


In [ ]:
real_df.to_csv("real_signal_df.csv")

In [ ]:
attack_df.to_csv("attack_signal_df.csv")

# Data Normalization

In [ ]:
def min_max_norm(df):
    x = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_minmax = pd.DataFrame(x_scaled)
    return df_minmax

In [ ]:
def std_norm(df):
    x = df.values
    std_scaler = preprocessing.StandardScaler()
    x_scaled = std_scaler.fit_transform(x)
    df_std = pd.DataFrame(x_scaled)
    return df_std

In [ ]:
x_superset = pd.concat( [std_norm(real_df), std_norm(attack_df)] , axis = 0 )

In [ ]:
y_superset = [1.0 for i in range(len(real_df)) ]
y_superset.extend([0.0 for i in range(len(attack_df))])

# Test Train split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_superset, y_superset, test_size=0.3, random_state = 1)

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

# Features extraction list

Features: PCA, Entropy, FFT Theta band, Bio Orthogonal Family, TF-IDF

models: SVM, Cosine, Kmeans, KNN, CNN

## PCA feature extraction

In [ ]:
best_feature_count = 100
pca = decomposition.PCA(n_components = best_feature_count) # only keep two "best" features!
X_train_pca = pca.fit_transform(X_train) # apply PCA to the train data
X_test_pca = pca.transform(X_test)

In [ ]:
type(X_test_pca)

numpy.ndarray

## Entropy

In [ ]:
def entropy_features(X):
    res_data=[]
    for i in range(len(X)):
        x = X[i]
                
        perm = ant.perm_entropy(x, normalize=True)
        spectral = ant.spectral_entropy(x, sf=160, method='welch', normalize=True)
        svd = ant.svd_entropy(x, normalize=True)
        approx = ant.app_entropy(x)
        sample = ant.sample_entropy(x)
        temp_ans = [perm, spectral, svd, approx, sample]
        res_data.append(temp_ans)
    return res_data

X_train_entropy = entropy_features(np.asarray(X_train))

X_test_entropy = entropy_features(np.asarray(X_test))


In [ ]:
X_train_entropy_array = np.asarray(X_train_entropy)
X_test_entropy_array = np.asarray(X_test_entropy)

X_test_entropy_array.shape











(954, 5)

In [ ]:
type (X_test_entropy_array)

numpy.ndarray

In [ ]:
X_train

pandas.core.frame.DataFrame

## FFT Beta band

In [ ]:

def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut /nyq
    high = highcut/nyq
    b, a = butter(order, [low, high], btype='band')
    #print(b,a)
    y = lfilter(b, a, data)
    return y
temp = X_train.values
temp1 = X_test.values

X_train_beta = []
for i in range(len(temp)):
    
    fft1 = fft (temp[i])
    beta1 = butter_bandpass_filter(fft1, 4, 8, 160)
    beta1 = [abs(x) for x in beta1]
    X_train_beta.append(beta1)

X_test_beta = []
for i in range(len(temp1)):
    
    fft1 = fft (temp1[i])
    beta1 = butter_bandpass_filter(fft1, 4, 8, 160)
    beta1 = [abs(x) for x in beta1]
    X_test_beta.append(beta1)

X_train_beta = np.asarray(X_train_beta)
X_test_beta = np.asarray(X_test_beta)


## Bio orthogonal family

In [ ]:
! pip install PyWavelets


You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pywt

X_train_list_bior = X_train.values.tolist()
X_test_list_bior = X_test.values.tolist()
X_train_bior = []
X_test_bior = []
for i in X_train_list_bior :
    x_1,x_2=pywt.dwt(i,'bior6.8')    
    X_train_bior.append(np.append(x_1,x_2))

for i in X_test_list_bior :
    x_1,x_2=pywt.dwt(i,'bior6.8')    
    X_test_bior.append(np.append(x_1,x_2))





In [ ]:
X_train_bior = np.asarray(X_train_bior)
X_test_bior = np.asarray (X_test_bior)


In [ ]:
import plotly.express as px

In [ ]:
px.line(X_train_bior[3])


In [ ]:
px.line(X_train.values[3])

## TF -IDF

In [ ]:
X_train_tfidf = X_train.values.tolist() 
X_train_tfidf_rounded = []
for i in range (len(X_train_tfidf)):
    temp = []
    for j in range (len(X_train_tfidf[i])):
        a = round(X_train_tfidf[i][j],2)
        temp.append (a)
    X_train_tfidf_rounded.append(temp)

X_train_tfidf_rounded_string = []
for i in range (len(X_train_tfidf_rounded)):
    temp = []
    for j in range (len(X_train_tfidf_rounded[i])):
        a = str(X_train_tfidf_rounded[i][j])
        temp.append (a)
    newtemp = " ".join(temp)
    X_train_tfidf_rounded_string.append(newtemp)

















In [ ]:
X_test_tfidf = X_test.values.tolist()
X_test_tfidf_rounded = []
for i in range (len(X_test_tfidf)):
    temp = []
    for j in range (len(X_test_tfidf[i])):
        a = round(X_test_tfidf[i][j],2)
        temp.append (a)
    X_test_tfidf_rounded.append(temp)
 
X_test_tfidf_rounded_string = []
for i in range (len(X_test_tfidf_rounded)):
    temp = []
    for j in range (len(X_test_tfidf_rounded[i])):
        a = str(X_test_tfidf_rounded[i][j])
        temp.append (a)
    newtemp = " ".join(temp)
    X_test_tfidf_rounded_string.append(newtemp)

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X_test_tfidf_rounded_string)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
X_test_tfidffeatures = pd.DataFrame (denselist, columns=feature_names)

C:\Users\prath\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X_train_tfidf_rounded_string)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
X_train_tfidffeatures = pd.DataFrame (denselist, columns=feature_names)

C:\Users\prath\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,01,02,03,04,05,06,07,08,09,10,...,89,91,92,93,94,95,96,97,98,99
0,0.108509,0.128983,0.133077,0.126935,0.135125,0.133077,0.141267,0.110556,0.106462,0.0,...,0.089827,0.052671,0.093456,0.067403,0.110431,0.072382,0.101983,0.085102,0.073513,0.083467
1,0.282291,0.214332,0.266608,0.231757,0.216074,0.209104,0.214332,0.181224,0.172511,0.0,...,0.050298,0.095771,0.091780,0.081954,0.071514,0.090355,0.053733,0.062085,0.062568,0.058504
2,0.122856,0.126951,0.122856,0.133094,0.112618,0.102380,0.104427,0.094189,0.133094,0.0,...,0.061468,0.093383,0.083881,0.091486,0.093638,0.077217,0.106853,0.089976,0.093128,0.095754
3,0.120606,0.095653,0.114368,0.122685,0.124765,0.139321,0.099812,0.116447,0.120606,0.0,...,0.100837,0.097265,0.102222,0.075793,0.075587,0.075966,0.076452,0.083966,0.069687,0.079788
4,0.116985,0.094409,0.090304,0.114932,0.094409,0.151875,0.110828,0.137508,0.106723,0.0,...,0.085308,0.081600,0.115305,0.082047,0.101075,0.096746,0.114404,0.077999,0.100714,0.113203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,0.191796,0.193677,0.161711,0.133505,0.165471,0.146668,0.163591,0.186155,0.154189,0.0,...,0.030395,0.028585,0.019808,0.019898,0.026458,0.022159,0.026761,0.015632,0.022506,0.011273
2222,0.187670,0.218702,0.224613,0.221658,0.218702,0.205403,0.200970,0.218702,0.180282,0.0,...,0.000000,0.001728,0.001730,0.001737,0.003465,0.000000,0.003505,0.000000,0.000000,0.003544
2223,0.197324,0.145501,0.143508,0.155467,0.145501,0.141515,0.155467,0.141515,0.115604,0.0,...,0.066738,0.039623,0.058323,0.058589,0.042069,0.035233,0.042551,0.042609,0.062025,0.043019
2224,0.222717,0.245389,0.233386,0.218716,0.234720,0.241388,0.217383,0.220050,0.232053,0.0,...,0.000000,0.000000,0.000000,0.000000,0.001564,0.001572,0.000000,0.000000,0.000000,0.000000


# Model Training section

##  MLP | PCA

In [ ]:
model_mlp = MLPClassifier( hidden_layer_sizes=(100), activation= 'relu', max_iter=1000, alpha=0.0001,
                           solver= 'adam', tol=0.0001 ) #changed from logistic to relu for better output
model_mlp.fit(X_train_pca, y_train) #train the data using the MLP                           
y_pred_mlp_pca = model_mlp.predict(X_test_pca) # predict the class of Test
accuracy_mlp_pca = accuracy_score(y_test, y_pred_mlp_pca) #accuracy of test data

In [ ]:
accuracy_mlp_pca

0.9758909853249476

In [ ]:
f1_score_mlp_pca = f1_score(y_test, y_pred_mlp_pca)
f1_score_mlp_pca

0.9714285714285713

## MLP | TFIDF

In [ ]:
model_mlp.fit(X_train_tfidffeatures, y_train) #train the data using the MLP
y_pred_mlp_tfidf = model_mlp.predict(X_test_tfidffeatures) # predict the class of Testa
accuracy_mlp_tfidf = accuracy_score(y_test, y_pred_mlp_tfidf) #accuracy of test data

accuracy_mlp_tfidf

0.6582809224318659

In [ ]:
f1_score_mlp_tfidf = f1_score(y_test, y_pred_mlp_tfidf)
f1_score_mlp_tfidf

0.5788113695090439

## MLP | Entropy

In [ ]:
model_mlp.fit(X_train_entropy_array, y_train) #train the data using the MLP
y_pred_mlp_entropy = model_mlp.predict(X_test_entropy_array) # predict the class of Testa
accuracy_mlp_entropy = accuracy_score(y_test, y_pred_mlp_entropy) #accuracy of test data

accuracy_mlp_entropy

C:\Users\prath\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:441: UserWarning:

X does not have valid feature names, but MLPClassifier was fitted with feature names



0.980083857442348

In [ ]:
f1_score_mlp_entropy = f1_score(y_test, y_pred_mlp_entropy)
f1_score_mlp_entropy

0.9758576874205844

## MLP | FFT Beta

In [ ]:
model_mlp.fit(X_train_beta, y_train) #train the data using the MLP
y_pred_mlp_beta = model_mlp.predict(X_test_beta) # predict the class of Testa
accuracy_mlp_beta = accuracy_score(y_test, y_pred_mlp_beta) #accuracy of test data

accuracy_mlp_beta

C:\Users\prath\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:441: UserWarning:

X does not have valid feature names, but MLPClassifier was fitted with feature names



0.9989517819706499

In [ ]:
f1_score_mlp_beta = f1_score(y_test, y_pred_mlp_beta)
f1_score_mlp_beta

0.9987577639751554

## MLP | Bior

In [ ]:
model_mlp.fit(X_train_bior, y_train) #train the data using the MLP
y_pred_mlp_bior = model_mlp.predict(X_test_bior) # predict the class of Testa
accuracy_mlp_bior = accuracy_score(y_test, y_pred_mlp_bior) #accuracy of test data

accuracy_mlp_bior

C:\Users\prath\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:441: UserWarning:

X does not have valid feature names, but MLPClassifier was fitted with feature names



0.850104821802935

In [ ]:
f1_score_mlp_bior = f1_score(y_test, y_pred_mlp_bior)
f1_score_mlp_bior

0.8258221680876979

## SVM | PCA

In [ ]:
model_svm = SVC(kernel='linear', C=1.0, random_state = 1 )
model_svm.fit(X_train_pca, y_train)     # do the training
y_pred_svm_pca = model_svm.predict(X_test_pca) # work on the test data

In [ ]:
accuracy_svm_pca = accuracy_score(y_test, y_pred_svm_pca) #accuracy of test data
accuracy_svm_pca

0.5953878406708596

In [ ]:
f1_score_svm_pca = f1_score(y_test, y_pred_svm_pca)
f1_score_svm_pca

0.07655502392344497

## SVM | Bior

In [ ]:
model_svm = SVC(kernel='linear', C=1.0, random_state = 1 )
model_svm.fit(X_train_bior, y_train)     # do the training
y_pred_svm_bior = model_svm.predict(X_test_bior) # work on the test data
accuracy_svm_bior = accuracy_score(y_test, y_pred_svm_bior) #accuracy of test data
accuracy_svm_bior


0.5303983228511531

In [ ]:
f1_score_svm_bior = f1_score(y_test, y_pred_svm_bior)
f1_score_svm_bior


0.4151436031331593

## SVM | TFIDF

In [ ]:
model_svm = SVC(kernel='linear', C=1.0, random_state = 1 )
model_svm.fit(X_train_tfidffeatures, y_train)     # do the training
y_pred_svm_tfidf = model_svm.predict(X_test_tfidffeatures) # work on the test data
accuracy_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf) #accuracy of test data
accuracy_svm_tfidf

0.5786163522012578

In [ ]:
f1_score_svm_tfidf = f1_score(y_test, y_pred_svm_tfidf)
f1_score_svm_tfidf

0.0

## SVM | Entropy

In [ ]:
model_svm = SVC(kernel='linear', C=1.0, random_state = 1 )
model_svm.fit(X_train_entropy, y_train)     # do the training
y_pred_svm_entropy = model_svm.predict(X_test_entropy) # work on the test data
accuracy_svm_entropy = accuracy_score(y_test, y_pred_svm_entropy) #accuracy of test data
accuracy_svm_entropy

0.9119496855345912

In [ ]:
f1_score_svm_entropy = f1_score(y_test, y_pred_svm_entropy)
f1_score_svm_entropy

0.8846153846153846

## SVM | FFT Beta

In [ ]:
model_svm = SVC(kernel='linear', C=1.0, random_state = 1 )
model_svm.fit(X_train_beta, y_train)     # do the training
y_pred_svm_beta = model_svm.predict(X_test_beta) # work on the test data
accuracy_svm_beta = accuracy_score(y_test, y_pred_svm_beta) #accuracy of test data
accuracy_svm_beta

0.9968553459119497

## KNN | PCA

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors = 25)
model_knn.fit(X_train_pca,y_train)
#Predict Output
y_pred_knn_pca = model_knn.predict(X_test_pca) 
accuracy_knn_pca = accuracy_score(y_test, y_pred_knn_pca) #accuracy of test data
accuracy_knn_pca

0.859538784067086

## KNN | Entropy

In [ ]:
model_knn_entropy = KNeighborsClassifier(n_neighbors = 25)
model_knn_entropy.fit(X_train_entropy_array,y_train)
#Predict Output
y_pred_knn_entropy = model_knn_entropy.predict(X_test_entropy_array) 
accuracy_knn_entropy = accuracy_score(y_test, y_pred_knn_entropy) #accuracy of test data
accuracy_knn_entropy



0.9863731656184487

## KNN | FFT Beta

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors = 25)
model_knn.fit(X_train_beta,y_train)
#Predict Output
y_pred_knn_beta = model_knn.predict(X_test_beta) 
accuracy_knn_beta = accuracy_score(y_test, y_pred_knn_beta) #accuracy of test data
accuracy_knn_beta

0.9968553459119497

## KNN | Bior

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors = 25)
model_knn.fit(X_train_bior,y_train)
#Predict Output
y_pred_knn_bior = model_knn.predict(X_test_bior) 
accuracy_knn_bior = accuracy_score(y_test, y_pred_knn_bior) #accuracy of test data
accuracy_knn_bior

0.6918238993710691

## KNN | TFIDF

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors = 25)
model_knn.fit(X_train_tfidffeatures,y_train)
#Predict Output
y_pred_knn_tfidf = model_knn.predict(X_test_tfidffeatures) 
accuracy_knn_tfidf = accuracy_score(y_test, y_pred_knn_tfidf) #accuracy of test data
accuracy_knn_tfidf

C:\Users\prath\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:441: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



0.6268343815513627

## CNN Classifier

In [ ]:
def cnn_evaluate(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 10, 32
	n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], trainy.shape[0]
	model = Sequential()
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
	model.add(Dropout(0.5))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy, model

In [ ]:
# y_train_array = np.asarray (y_train)
# y_test_array = np.asarray (y_test)

X_train_cnn = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

#cnn_accuracy = cnn_evaluate(X_train_pca.values, to_categorical(y_train_array), X_test_pca, to_categorical (y_test_array))

# cnn_accuracy

In [ ]:
print("X Train shape: ", X_train_cnn.shape)
print("X Test shape: ", X_test_cnn.shape)

X Train shape:  (2226, 4800, 1)
X Test shape:  (954, 4800, 1)


In [ ]:
# Create sequential model 
cnn_model = tf.keras.models.Sequential()
#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (X_train_cnn.shape[1],1)))
#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Third CNN layer with 128 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=128, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Fourth CNN layer with Max pooling
cnn_model.add(MaxPooling1D(pool_size=(3,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))
#Flatten the output
cnn_model.add(Flatten())
#Add a dense layer with 256 neurons
cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 2, activation='softmax'))



In [ ]:
cnn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 4800, 32)          128       
                                                                 
 conv1d_10 (Conv1D)          (None, 4800, 64)          6208      
                                                                 
 conv1d_11 (Conv1D)          (None, 4800, 128)         24704     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 2400, 128)        0         
 1D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 2400, 128)         0         
                                                                 
 flatten_3 (Flatten)         (None, 307200)            0         
                                                      

In [ ]:
y_train_cnn = to_categorical(y_train)
y_test_cnn = to_categorical(y_test)
cnn_model_history = cnn_model.fit(X_train_cnn, np.asarray(y_train), epochs=2, batch_size = 20 , validation_data = (X_test_cnn, np.asarray (y_test)))

Epoch 1/2
112/112 [==============================] - 110s 977ms/step - loss: 0.3368 - accuracy: 0.8953 - val_loss: 0.1260 - val_accuracy: 0.9654
Epoch 2/2
112/112 [==============================] - 110s 982ms/step - loss: 0.0170 - accuracy: 0.9969 - val_loss: 0.0103 - val_accuracy: 0.9979


In [ ]:
def train_cnn_model(X_train, X_test, y_train, y_test, epochs = 2):
    X_train_cnn = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test_cnn = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
    # Create sequential model 
    cnn_model = tf.keras.models.Sequential()
    #First CNN layer  with 32 filters, conv window 3, relu activation and same padding
    cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (X_train_cnn.shape[1],1)))
    #Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
    cnn_model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Third CNN layer with 128 filters, conv window 3, relu activation and same padding
    cnn_model.add(Conv1D(filters=128, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Fourth CNN layer with Max pooling
    cnn_model.add(MaxPooling1D(pool_size=(3,), strides=2, padding='same'))
    cnn_model.add(Dropout(0.5))
    #Flatten the output
    cnn_model.add(Flatten())
    #Add a dense layer with 256 neurons
    cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Add a dense layer with 512 neurons
    cnn_model.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Softmax as last layer with five outputs
    cnn_model.add(Dense(units = 2, activation='softmax'))
    cnn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
    y_train_cnn = to_categorical(y_train)
    y_test_cnn = to_categorical(y_test)
    cnn_model_history = cnn_model.fit(X_train_cnn, np.asarray(y_train), epochs=2, batch_size = 20 , validation_data = (X_test_cnn, np.asarray (y_test)))
    return cnn_model_history
        

## CNN | PCA

In [ ]:
cnn_pca = train_cnn_model(X_train_pca, X_test_pca, y_train, y_test)

Epoch 1/2
112/112 [==============================] - 8s 66ms/step - loss: 0.2753 - accuracy: 0.9385 - val_loss: 0.0736 - val_accuracy: 0.9822
Epoch 2/2
112/112 [==============================] - 3s 25ms/step - loss: 0.0778 - accuracy: 0.9811 - val_loss: 0.0536 - val_accuracy: 0.9864


In [ ]:
cnn_pca.history

{'loss': [0.27527302503585815, 0.07784483581781387],
 'accuracy': [0.9384546279907227, 0.9811320900917053],
 'val_loss': [0.07359401881694794, 0.05357757583260536],
 'val_accuracy': [0.9821802973747253, 0.9863731861114502]}

## CNN | ENtropy

In [ ]:
cnn_entropy = train_cnn_model(X_train_entropy_array, X_test_entropy_array, y_train, y_test)

Epoch 1/2
112/112 [==============================] - 2s 9ms/step - loss: 0.4856 - accuracy: 0.7583 - val_loss: 0.2374 - val_accuracy: 0.9182
Epoch 2/2
112/112 [==============================] - 1s 9ms/step - loss: 0.1867 - accuracy: 0.9429 - val_loss: 0.1268 - val_accuracy: 0.9612


In [ ]:
cnn_entropy.history

{'loss': [0.485586941242218, 0.1867331862449646],
 'accuracy': [0.758310854434967, 0.9429469704627991],
 'val_loss': [0.23742970824241638, 0.12680864334106445],
 'val_accuracy': [0.9182389974594116, 0.9612159132957458]}

## CNN| FFT beta

In [ ]:
cnn_beta = train_cnn_model(X_train_beta, X_test_beta, y_train, y_test)

Epoch 1/2
112/112 [==============================] - 115s 1s/step - loss: 1.2257 - accuracy: 0.9721 - val_loss: 0.0235 - val_accuracy: 0.9990
Epoch 2/2
112/112 [==============================] - 117s 1s/step - loss: 0.0022 - accuracy: 0.9996 - val_loss: 0.0071 - val_accuracy: 0.9990


## CNN| BIor

In [ ]:
cnn_bior = train_cnn_model(X_train_bior, X_test_bior, y_train, y_test, epcohs = 10)

Epoch 1/2
112/112 [==============================] - 119s 1s/step - loss: 0.4222 - accuracy: 0.9236 - val_loss: 0.0057 - val_accuracy: 0.9979
Epoch 2/2
112/112 [==============================] - 110s 979ms/step - loss: 0.0039 - accuracy: 0.9996 - val_loss: 0.0472 - val_accuracy: 0.9853


## CNN | TFIDF

In [ ]:
cnn_tfidf = train_cnn_model(X_train_tfidffeatures, X_test_tfidffeatures, y_train, y_test, epochs= 10)

Epoch 1/2
112/112 [==============================] - 3s 26ms/step - loss: 0.6512 - accuracy: 0.6074 - val_loss: 0.6475 - val_accuracy: 0.5786
Epoch 2/2
112/112 [==============================] - 3s 25ms/step - loss: 0.6340 - accuracy: 0.6092 - val_loss: 0.6467 - val_accuracy: 0.5786


## K Means | PCA

In [ ]:
n_clusters = len(np.unique(y_train))
kmeans_model = KMeans(n_clusters = n_clusters, random_state=42)
kmeans_model.fit(X_train_pca)
y_labels_train_kmeans_pca = kmeans_model.labels_
y_labels_test_kmeans_pca = kmeans_model.predict(X_test_pca)

# X_train['km_clust'] = y_labels_train
# X_test['km_clust'] = y_labels_test

accuracy_kmeans = accuracy_score(y_train, y_labels_train_kmeans_pca)
accuracy_kmeans = accuracy_score (y_test, y_labels_test_kmeans_pca)
accuracy_kmeans 





0.5796645702306079

## K Means | Entropy

In [ ]:
n_clusters = len(np.unique(y_train))
kmeans_model = KMeans(n_clusters = n_clusters, random_state=42)
kmeans_model.fit(X_train_entropy)
y_labels_train_kmeans_entropy = kmeans_model.labels_
y_labels_test_kmeans_entropy = kmeans_model.predict(X_test_entropy)

# X_train['km_clust'] = y_labels_train
# X_test['km_clust'] = y_labels_test

accuracy_kmeans_entropy = accuracy_score(y_train, y_labels_train_kmeans_entropy)
accuracy_kmeans_entropy = accuracy_score (y_test, y_labels_test_kmeans_entropy)
accuracy_kmeans_entropy 





0.48742138364779874

## K Means | FFT - Beta

In [ ]:
n_clusters = len(np.unique(y_train))
kmeans_model = KMeans(n_clusters = n_clusters, random_state=42)
kmeans_model.fit(X_train_beta)
y_labels_train_kmeans_beta = kmeans_model.labels_
y_labels_test_kmeans_beta = kmeans_model.predict(X_test_beta)
accuracy_kmeans_beta = accuracy_score(y_train, y_labels_train_kmeans_beta)
accuracy_kmeans_beta = accuracy_score (y_test, y_labels_test_kmeans_beta)
accuracy_kmeans_beta





0.4790356394129979

## K Means | Biorthogonal

In [ ]:
n_clusters = len(np.unique(y_train))
kmeans_model = KMeans(n_clusters = n_clusters, random_state=42)
kmeans_model.fit(X_train_bior)
y_labels_train_kmeans_bior = kmeans_model.labels_
y_labels_test_kmeans_bior = kmeans_model.predict(X_test_bior)
accuracy_kmeans_bior = accuracy_score(y_train, y_labels_train_kmeans_bior)
accuracy_kmeans_bior = accuracy_score (y_test, y_labels_test_kmeans_bior)
accuracy_kmeans_bior





0.4171907756813417

## K Means | TF IDF 

In [ ]:
n_clusters = len(np.unique(y_train))
kmeans_model = KMeans(n_clusters = n_clusters, random_state=42)
kmeans_model.fit(X_train_tfidffeatures)
y_labels_train_kmeans_tfidf = kmeans_model.labels_
y_labels_test_kmeans_tfidf = kmeans_model.predict(X_test_tfidffeatures)
accuracy_kmeans_tfidf = accuracy_score(y_train, y_labels_train_kmeans_tfidf)
accuracy_kmeans_tfidf = accuracy_score (y_test, y_labels_test_kmeans_tfidf)
accuracy_kmeans_tfidf





0.5482180293501048

In [ ]:
nm = (f"{y_pred_knn_beta=}").split("=")[0]

nm

'y_pred_knn_beta'

## F1 scores list

In [ ]:
f1_preds = [y_pred_mlp_pca, y_pred_mlp_tfidf, y_pred_mlp_entropy, y_pred_mlp_beta, y_pred_mlp_bior,y_pred_svm_pca, y_pred_svm_bior, y_pred_svm_beta,y_pred_svm_tfidf, y_pred_svm_entropy,y_pred_knn_pca,y_pred_knn_entropy,y_pred_knn_beta, y_pred_knn_bior, y_pred_knn_tfidf,y_labels_test_kmeans_pca,y_labels_test_kmeans_beta,y_labels_test_kmeans_bior,y_labels_test_kmeans_tfidf, y_labels_test_kmeans_entropy]
f1_preds = [y_pred_mlp_pca, y_pred_mlp_tfidf, y_pred_mlp_entropy, y_pred_mlp_beta, y_pred_mlp_bior,y_pred_svm_pca, y_pred_svm_bior, y_pred_svm_beta,y_pred_svm_tfidf, y_pred_svm_entropy,y_pred_knn_pca,y_pred_knn_entropy,y_pred_knn_beta, y_pred_knn_bior, y_pred_knn_tfidf,y_labels_test_kmeans_pca,y_labels_test_kmeans_beta,y_labels_test_kmeans_bior,y_labels_test_kmeans_tfidf, y_labels_test_kmeans_entropy]

for i in f1_preds :
    temp = f1_score(y_test, i)
    print ("\nf1 score of ",(f"{i=}").split("=")[0]," is ", temp)

    


f1 score of  i  is  0.9714285714285713

f1 score of  i  is  0.5788113695090439

f1 score of  i  is  0.9758576874205844

f1 score of  i  is  0.9987577639751554

f1 score of  i  is  0.8258221680876979

f1 score of  i  is  0.07655502392344497

f1 score of  i  is  0.4151436031331593

f1 score of  i  is  0.996282527881041

f1 score of  i  is  0.0

f1 score of  i  is  0.8846153846153846

f1 score of  i  is  0.8017751479289941

f1 score of  i  is  0.9838107098381071

f1 score of  i  is  0.9962732919254659

f1 score of  i  is  0.42352941176470593

f1 score of  i  is  0.531578947368421

f1 score of  i  is  0.004962779156327543

f1 score of  i  is  0.0

f1 score of  i  is  0.588757396449704

f1 score of  i  is  0.5835748792270531

f1 score of  i  is  0.47135135135135137


## FAR FRR

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return FP/(FP + TN), FN/(FN + TP)




In [ ]:
for i in f1_preds :
   far, frr = perf_measure (y_test, i) 
   print ("far, frr : ", far,",",frr)

far, frr :  0.021739130434782608 , 0.02736318407960199
far, frr :  0.26811594202898553 , 0.4427860696517413
far, frr :  0.0018115942028985507 , 0.04477611940298507
far, frr :  0.0018115942028985507 , 0.0
far, frr :  0.14492753623188406 , 0.15671641791044777
far, frr :  0.0 , 0.9601990049751243
far, frr :  0.3713768115942029 , 0.6044776119402985
far, frr :  0.005434782608695652 , 0.0
far, frr :  0.0 , 1.0
far, frr :  0.007246376811594203 , 0.19900497512437812
far, frr :  0.005434782608695652 , 0.32587064676616917
far, frr :  0.010869565217391304 , 0.017412935323383085
far, frr :  0.0036231884057971015 , 0.0024875621890547263
far, frr :  0.0 , 0.7313432835820896
far, frr :  0.2826086956521739 , 0.4975124378109453
far, frr :  0.0 , 0.9975124378109452
far, frr :  0.1721014492753623 , 1.0
far, frr :  1.0 , 0.009950248756218905
far, frr :  0.5996376811594203 , 0.24875621890547264
far, frr :  0.552536231884058 , 0.4577114427860697


# Timepass

In [ ]:
x=real_df.values[0]

In [ ]:
x_ = attack_df.values[0]

In [ ]:
px.line(x[:1000])

In [ ]:
px.line(x_superset [0][:1000])

In [ ]:
x_superset [0]

In [ ]:
z_ = scipy.io.loadmat('data/inputSample.mat')

FileNotFoundError: [Errno 2] No such file or directory: 'data/inputSample.mat'

In [ ]:
z_['data'].shape

(1, 4800)

In [ ]:
a_ = scipy.io.loadmat('data/input_attack.mat')

In [ ]:
a_['data'].shape

(1, 4800)

In [ ]:
sd1 = [[1,2,3],[4,5,6]]
sd2 = [[7,8]]
sd1_df = pd.DataFrame(sd1)
sd2_df = pd.DataFrame(sd2)

In [ ]:
sd1_df.transpose()

,0,1
0,1,4
1,2,5
2,3,6


In [ ]:
sd2_df.transpose()

,0
0,7
1,8


In [ ]:
pd.concat([sd1_df.transpose(), sd2_df.transpose()], axis=0)

,0,1
0,1,4.0
1,2,5.0
2,3,6.0
0,7,NaN
1,8,NaN


In [ ]:



# Data laoding  ------------------------------------------------
def load_realsignal_from_path(fpath):  #load real signal from path
    mat = scipy.io.loadmat(fpath)
    raw_data = mat['Raw_Data']
    total_rows = []
    for i in range(raw_data.shape[0]):
        for j in range(raw_data.shape[1]):
            total_rows.extend(np.array_split(raw_data[i][j], 4))
    raw_df = pd.DataFrame(total_rows)
    return raw_df
 
def load_attacksignal_from_path(fpath): #load attack signal from path
    mat = scipy.io.loadmat(fpath)
    raw_data = mat['attackVectors']
    total_rows = []
    for i in range(raw_data.shape[0]):
        for j in range(raw_data.shape[1]):
            for k in range(raw_data.shape[2]):
                total_rows.append(raw_data[i][j][k])
    raw_df = pd.DataFrame(total_rows)
    return raw_df

def load_inputsignal_from_path(fpath):  #load real signal from path
    mat = scipy.io.loadmat(fpath)
    raw_data = mat['data']
    
    total_rows = []
    for i in range(raw_data.shape[0]):
        
                total_rows.append(raw_data[i])
    raw_df = pd.DataFrame(total_rows)
    return raw_df


# ## NORM functions --
# def std_norm(df):
#     x = df.values
#     std_scaler = preprocessing.StandardScaler()
#     x_scaled = std_scaler.fit_transform(x)
#     df_std = pd.DataFrame(x_scaled)
#     return df_std

# real_signal_filename = 'Dataset1.mat'
# attack_signal_filename = 'sampleAttack.mat'
# b

# real_df = load_realsignal_from_path(real_signal_filename)
# attack_df = load_attacksignal_from_path(attack_signal_filename)
# input_df = load_inputsignal_from_path(input_signal_filenme)


# x_superset = pd.concat( [std_norm(real_df), std_norm(attack_df), std_norm(input_df)] , axis = 0 )
# y_superset = [1.0 for i in range(len(real_df)) ]
# y_superset.extend([0.0 for i in range(len(attack_df))])

# # Test train split ----------------------------------------------------


# # Feature extraction --------------------------------------------------

# ## PCA -----------
# best_feature_count = 100
# pca = decomposition.PCA(n_components = best_feature_count) # only keep two "best" features!
# X_pca = pca.fit_transform(x_superset) # apply PCA to the train data
# input_pca = X_pca[-1]

# ## TFIDF -------------

# X_tfidf = x_superset.values.tolist()
# X_tfidf_rounded = []
# for i in range (len(X_tfidf)):
#     temp = []
#     for j in range (len(X_tfidf[i])):
#         a = round(X_tfidf[i][j],2)
#         temp.append (a)
#     X_tfidf_rounded.append(temp)

# X_tfidf_rounded_string = []
# for i in range (len(X_tfidf_rounded)):
#     temp = []
#     for j in range (len(X_tfidf_rounded[i])):
#         a = str(X_tfidf_rounded[i][j])
#         temp.append (a)
#     newtemp = " ".join(temp)
#     X_tfidf_rounded_string.append(newtemp)


# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(X_tfidf_rounded_string)
# feature_names = vectorizer.get_feature_names()
# dense = vectors.todense()
# denselist = dense.tolist()
# X_tfidffeatures = pd.DataFrame (denselist, columns=feature_names)

# input_tfidf = X_tfidffeatures.values[-1]

# ## Entropy ------

def entropy_features1(X):
    res_data=[]
    for i in range(len(X)):
        x = X[i]
                
        perm = ant.perm_entropy(x, normalize=True)
        spectral = ant.spectral_entropy(x, sf=160, method='welch', normalize=True)
        svd = ant.svd_entropy(x, normalize=True)
        approx = ant.app_entropy(x)
        sample = ant.sample_entropy(x)
        temp_ans = [perm, spectral, svd, approx, sample]
        res_data.append(temp_ans)
    return res_data


# input_entropy = entropy_features(std_norm(input_df).values[0])

# print (input_entropy)
# print (input_pca)
# print (input_tfidf)

real_signal_filename = 'Dataset1.mat'

real_df = load_realsignal_from_path(real_signal_filename)



# LOAD MODELS -----------------------------------------------------------------------

In [ ]:
input_signal_filenme = 'inputSample.mat'
attack_signal_filename = 'sampleAttack.mat'
real_signal_filename = 'Dataset1.mat'
real_df = load_realsignal_from_path (real_signal_filename)
input_df = load_inputsignal_from_path(input_signal_filenme)
attack_df = load_attacksignal_from_path (attack_signal_filename)
x_superset = pd.concat( [(real_df), (attack_df), (input_df)] , axis = 0 )
x_superset.shape

[[ 5.34988656 -2.15044386 -8.25538683 ...  2.82498143 14.04324851
  23.55405672]] (1, 4800)


(3181, 4800)

In [ ]:
type(x_superset)
x_superset = std_norm (x_superset)

entropyop = entropy_features1 (x_superset)




In [ ]:
input_entropy = entropy_features(std_norm(input_df).values[0])

D:\Python\lib\site-packages\antropy\entropy.py:248: RuntimeWarning:

invalid value encountered in true_divide

D:\Python\lib\site-packages\antropy\entropy.py:361: RuntimeWarning:

invalid value encountered in true_divide



In [ ]:
# print (input_entropy)
# print (input_pca)
print (input_tfidf)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
X_train_entropy[0]

[0.6647610154249937,
 0.5194487259147607,
 0.6148811041017952,
 0.6847433140062855,
 0.6370126038238632]